# OpenMC Example

## Create a full core configuration
    Assemblies are 17x17 lattices of UO2 fuel pins
    Total Volume of 32.63 m^3
    Thermal Power of 3.3 GW

## Fuel Pins:
    Pitch of 1.4 cm between each pin

    1.5% Enriched Uranium
    UO2 Density = 10.4 g/cm3
    Fuel outer radius = 0.5207 cm

    Helium Gap Fill
    He Density = 0.125 g/cm3
    Gap outer radius = 0.53213 cm

    Pure Zirconium Cladding
    Zr Density = 6.0 g/cm3
    Clad outer radius = 0.61341 cm

    External Water Coolant
    

## Import Modules
    numpy for math functions/constants
    matplotlib.pyplot for visualization
    openmc
    openmc.deplete for depletion

In [ ]:
%matplotlib inline
from IPython.display import Image
import os
os.environ['PATH'] += os.pathsep + '/workspaces/codespaces-blank/openmc-workshop/openmc/release/bin/'
import numpy as np
import matplotlib.pyplot as plt
import openmc
import openmc.deplete

## Material Definitions
    UO2 : 1.5% Enrichment, 10.4 g/cm3 !!! O in UO2 S(α,β) !!!
    He  : 0.125 g/cm3
    Zr  : 6.0 g/cm3
    H2O : 1.00 g/cm3, !!! H in H2O S(α,β) !!!
    

In [ ]:
openmc.Materials.cross_sections = '/workspaces/codespaces-blank/openmc-workshop/endfb-viii.0-hdf5/cross_sections.xml'

fuel = openmc.Material()
fuel.add_element('U',1,enrichment = 1.5)
fuel.add_element('O',2)
fuel.add_s_alpha_beta('c_O_in_UO2')
fuel.set_density('g/cm3', 10.4)

helium = openmc.Material()
helium.add_element('He',1)
helium.set_density('g/cm3',0.125)

clad = openmc.Material()
clad.add_element('Zr',1)
clad.set_density('g/cm3',6.0)

water = openmc.Material()
#water.add_nuclide('H1',2,'wo') #For adding specific nuclides
water.add_element('O',1)
water.add_s_alpha_beta('c_H_in_H2O')
water.set_density('g/cm3',1.00)

materials = openmc.Materials([fuel, helium, clad, water])


## Geometry Definitions
    Pin Radii [0.52070, 0.53213, 0.61341] cm
    Assembly = 17x17 pins, pitch = 1.4
    Core = 15x15 circle of assemblies
    

In [ ]:
radii = [0.52070, 0.53213, 0.61341]
colors = {fuel : 'red', helium : 'white', clad : 'grey', water : 'blue'}
surf = [openmc.ZCylinder(r = r) for r in radii]

fuel_region = -surf[0]
gap_region = +surf[0] & -surf[1]
clad_region = +surf[1] & -surf[2]
water_region = +surf[2]

fuel_cell = openmc.Cell(fill = fuel, region = fuel_region)
gap_cell = openmc.Cell(fill = helium, region = gap_region)
clad_cell = openmc.Cell(fill = clad, region = clad_region)
water_cell = openmc.Cell(fill = water, region = water_region)

pin = openmc.Universe(cells = [fuel_cell, gap_cell, clad_cell, water_cell])
#pin.plot(width = (2.5,2.5),color_by = 'material', colors = colors)

pitch = 1.4
quarter_pitch = 8.5*pitch
assembly_width = 18*pitch
lattice = [[pin for i in range(17)]]*17

assembly = openmc.RectLattice()
assembly.universes = lattice
assembly.pitch = [pitch, pitch]
assembly.lower_left = [-quarter_pitch,-quarter_pitch]
assembly.outer = openmc.Universe(cells = [openmc.Cell(fill = water)])

assembly_univ = openmc.Universe(cells = [openmc.Cell(fill = assembly)])
assembly_bounds = openmc.model.RectangularPrism(width = assembly_width, height = assembly_width, axis = 'z')
void_univ = openmc.Universe(cells = [openmc.Cell(fill = water, region = -assembly_bounds)])

#assembly_univ.plot(width = (assembly_width,assembly_width), color_by = 'material',colors = colors)

core_lattice = [[[] for i in range(15)] for j in range(15)]
for i in range(15):
    for j in range(15):
        loc1 = i - 7
        loc2 = j - 7
        if loc1*loc1 + loc2*loc2 <= 53:
            core_lattice[i][j] = assembly_univ 
        else:
            core_lattice[i][j] = void_univ

core = openmc.RectLattice()
core.universes = core_lattice
core.pitch = [17.5*pitch, 17.5*pitch]
core.lower_left = [-7.5*17.5*pitch,-7.5*17.5*pitch]
core.outer = openmc.Universe(cells = [openmc.Cell(fill = water)])

core_bounds = openmc.model.RectangularPrism(width = 15*17.5*pitch, height = 15*17.5*pitch, axis = 'z', boundary_type = 'vacuum')
core_univ = openmc.Universe(cells = [openmc.Cell(fill = core, region = -core_bounds)])
#core_univ.plot(width = (15*18*pitch,15*18*pitch),color_by = 'material',colors = colors, pixels = (1000, 1000))

geometry = openmc.Geometry(core_univ)


## Tally Definitions
    Add a 1000x1000 tally mesh filter
    Add a 500 bin energy filter between 0.1 meV and 10 MeV

    Implement a Fission, Absorption, and Flux tally
    Normalize with respect to a κ-fission tally

In [ ]:
tallies = openmc.Tallies()

mesh = openmc.RegularMesh()
mesh.dimension = [1000, 1000]
mesh.lower_left = [-7.5*17.5*pitch,-7.5*17.5*pitch]
mesh.upper_right = [7.5*17.5*pitch,7.5*17.5*pitch]

mesh_filter = openmc.MeshFilter(mesh)

fission = openmc.Tally()
fission.scores = ['fission']
fission.filters = [mesh_filter]

absorption = openmc.Tally()
absorption.scores = ['absorption']
absorption.filters = [mesh_filter]

energy_filter = openmc.EnergyFilter(np.logspace(-4, 7, 501))

flux = openmc.Tally()
flux.scores = ['flux']
flux.filters = [energy_filter]

norm = openmc.Tally()
norm.scores = ['kappa-fission']

for tally in [fission, absorption, flux, norm]:
    tallies.append(tally)


## Settings
    Initialize a random seed with 2^31 - 1
    Use 300 total batches
    Use 90 inactive batches
    Use 10,000 particles

    add an entropy mesh of size [60, 60]

    export to a model
    

In [ ]:
settings = openmc.Settings()
settings.seed = np.random.randint(2**31 - 1)*2 + 1
settings.batches = 300
settings.inactive = 90
settings.particles = 10_000

entropy_mesh = openmc.RegularMesh()
entropy_mesh.dimension = [60, 60]
entropy_mesh.lower_left = [-7.5*17.5*pitch,-7.5*17.5*pitch]
entropy_mesh.upper_right = [7.5*17.5*pitch,7.5*17.5*pitch]

settings.entropy_mesh = entropy_mesh

model = openmc.model.Model(materials = materials, geometry = geometry, tallies = tallies, settings = settings)
model.export_to_xml()

openmc.run()

## Postprocessing
    Total Reactor Volume = 32.63 m3
    Average Energy per Fission = 190 MeV
    
### Get Tallies from statepoint
    .get_tally(name = tally_name).get_reshaped_data().reshape((shape))
    
### Normalize all tallies
    f = (Power)/(norm) [particle / s]
### Visualize Fission and Absorption Tallies over space [reactions / particle] -> [reactions / s]
    reaction rate = (f)*(reaction_tally)
### Visualize Flux Tally over energy [neutron * cm / particle] -> [neutron / cm2 / s]
    flux = (f)*(flux_tally)/(Volume)

In [ ]:
sp = openmc.StatePoint('statepoint.300.h5')

fission = sp.get_tally(scores = ['fission']).get_reshaped_data().reshape((1000,1000))
absorption = sp.get_tally(scores = ['absorption']).get_reshaped_data().reshape((1000,1000))
flux = sp.get_tally(scores = ['flux']).get_reshaped_data().flatten()
norm = sp.get_tally(scores = ['kappa-fission']).get_reshaped_data().flatten()

f = 3.3e9/(1.602e-19*norm[0])

fission *= f
absorption *= f

flux *= f/(32.63e6)

entropy = sp.entropy

plt.plot(entropy)

plt.figure()
plt.semilogx(np.logspace(-4,7,500),flux)
plt.grid()
plt.xlabel('Energy (eV)')
plt.ylabel(r'flux $\frac{\rm n}{\rm cm^2s}$')

## Depletion
    Use a CoupledOperator with the 'simple_chain.xml'
    Power Total Power = 3.3 GW
    Fuel Volume = 1.7428e7 cm3

    10 timesteps of 12 hours
    Use a PredictorIntegrator

In [ ]:
fuel.volume = 1.7428e7
materials = openmc.Materials([fuel, helium, clad, water])

model = openmc.model.Model(materials = materials, geometry = geometry, tallies = tallies, settings = settings)

operator = openmc.deplete.CoupledOperator(model, './simple_chain.xml')
power = 3.3e9
timesteps = [0.5]*10

integrator = openmc.deplete.PredictorIntegrator(operator, timesteps, power, timestep_units = 'd')

integrator.integrate()

# Depletion Results
## Get results from Results file
    openmc.deplete.Results

    time, keff = results.get_keff()
    time /= 86400

    _, isotope = results.get_atoms('1','AnZ')

## Plot Xe135, I135, Pm149, and Sm149

In [ ]:
results = opencm.deplete.Results('depletion_results.h5')
time, k = results.get_keff()
time /= 86400 # Seconds in a Day

_, xe135 = results.get_atoms('1', 'Xe135')
_, i135 = results.get_atoms('1', 'I135')
_, pm149 = results.get_atoms('1', 'Pm149')
_, sm149 = results.get_atoms('1', 'Sm149')

plt.figure()
plt.grid()
plt.plot(time, xe135, label = r'$^{135}$Xe')
plt.plot(time, i135, label = r'$^{135}$I')
plt.plot(time, pm135, label = r'$^{149}$Pm')
plt.plot(time, sm135, label = r'$^{149}$Sm')
plt.legend()
